In [2]:
def f(x):
    return x**4 - 3 * x

def integrate_f(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [3]:
integrate_f(f, -10, +10, 1_000_000)

39999.999999868494

In [4]:
%timeit integrate_f(f, -10, +10, 1_000_000)

483 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
%load_ext cython

In [6]:
%%cython

def f2(x):
    return x**4 - 3 * x

def integrate_f2(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [8]:
integrate_f2(f2, -10, +10, 1_000_000)

39999.999999868494

In [9]:
%timeit integrate_f2(f2, -10, +10, 1_000_000)

394 ms ± 8.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%cython
#Type specialization
#Not python anymore, only Cython
def f3(double x):
    return x**4 - 3 * x

def integrate_f3(func, double a, double b, int n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

Content of stdout:
_cython_magic_fc41c0d6f3dcf7423c61da3722e5d775ad6b451b.c
   Creando biblioteca C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_fc41c0d6f3dcf7423c61da3722e5d775ad6b451b.cp311-win_amd64.lib y objeto C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_fc41c0d6f3dcf7423c61da3722e5d775ad6b451b.cp311-win_amd64.exp
Generando c¢digo
Generaci¢n de c¢digo finalizada

In [11]:
integrate_f3(f3, -10, +10, 1_000_000)

39999.999999868494

In [12]:
%timeit integrate_f3(f3, -10, +10, 1_000_000)

237 ms ± 8.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%cython -a
#type of internal variables

def f4(double x):
    return x**4 - 3 * x

def integrate_f4(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

Content of stdout:
_cython_magic_2256fc8b99f2067bb5d75895532b2dd4e12dd26c.c
   Creando biblioteca C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_2256fc8b99f2067bb5d75895532b2dd4e12dd26c.cp311-win_amd64.lib y objeto C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_2256fc8b99f2067bb5d75895532b2dd4e12dd26c.cp311-win_amd64.exp
Generando c¢digo
Generaci¢n de c¢digo finalizada

In [18]:
integrate_f4(f4, -10, +10, 1_000_000)

39999.999999868494

In [17]:
%timeit integrate_f4(f4, -10, +10, 1_000_000)


130 ms ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%cython

# We do "function specialization", by changing 'f5' to be a C function.
#
# On the first attempt we get a warning, because the generated C function is not used.

cdef double f5(double x):
    return x**4 - 3 * x

def integrate_f5(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [32]:
integrate_f5(f5, -10, +10, 1_000_000)

NameError: name 'f5' is not defined

In [33]:
%%cython -a

# In the second attempt, integrate_f5 hardcodes the function to call.
# We can rename it to just 'f', because it'll not be visible anywhere.

cdef double f(double x):
    return x**4 - 3 * x

def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [34]:
integrate_f5(-10, +10, 1_000_000)


39999.999999868494

In [35]:
%timeit integrate_f5(-10, +10, 1_000_000)

51.2 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%cython

cdef double f(double x):
    return (x*x*x - 3) * x

def integrate_f6(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [37]:
integrate_f6(-10, +10, 1_000_000)

39999.999999868494

In [38]:
%timeit integrate_f6(-10, +10, 1_000_000)

2.51 ms ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1]:
%load_ext cython

In [17]:
%%cython
# Definir la función de la EDO dy/dx = x + y
cdef double f(double x, double y):
    return x + y

def runge_kutta_4(double x0, double y0, double x_end, double step_size):
    """
    Resuelve una ecuación diferencial ordinaria utilizando el método de Runge-Kutta de cuarto orden (RK4).
    
    Parámetros:
    - f: La función de la EDO de la forma f(x, y).
    - x0: El valor inicial de x.
    - y0: El valor inicial de y (condición inicial).
    - x_end: El valor final de x hasta donde se quiere aproximar la solución.
    - step_size: El tamaño del paso (h).
    
    Retorna:
    - La aproximación de y en x_end.
    """
    cdef:
        double x = x0
        double y = y0

    # Iterar hasta alcanzar x_end
    while x < x_end:
        k1 = step_size * f(x, y)
        k2 = step_size * f(x + step_size / 2, y + k1 / 2)
        k3 = step_size * f(x + step_size / 2, y + k2 / 2)
        k4 = step_size * f(x + step_size, y + k3)
        
        y += (k1 + 2 * k2 + 2 * k3 + k4) / 6
        x += step_size

    return y

Content of stdout:
_cython_magic_7b882ae1346df9d5ba3ed437b289cdd5aac3b902.c
   Creando biblioteca C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_7b882ae1346df9d5ba3ed437b289cdd5aac3b902.cp311-win_amd64.lib y objeto C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_7b882ae1346df9d5ba3ed437b289cdd5aac3b902.cp311-win_amd64.exp
Generando c¢digo
Generaci¢n de c¢digo finalizada

In [19]:
runge_kutta_4(0, 1, 2, 0.1)

11.778089534751086

In [88]:
#no cython
%timeit runge_kutta_4(f, 0, 1, 2, 0.1)

11.9 µs ± 92.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [90]:
##sólo cython
%timeit runge_kutta_4(f, 0, 1, 2, 0.1)

6.5 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
##variable specialization
%timeit runge_kutta_4(f, 0, 1, 2, 0.1)

6.09 µs ± 84.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
##internal variable
%timeit runge_kutta_4(f, 0, 1, 2, 0.1)

6.3 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [21]:
##function specialization
%timeit runge_kutta_4(0, 1, 2, 0.1)

356 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [48]:
def f(x, y):
    return x**3 - 2*y**2 + 4

def runge_kutta_4(f, x0, y0, x_end, step_size):
    """ Resuelve una ecuación diferencial ordinaria utilizando el método de Runge-Kutta de cuarto orden (RK4).
    
    Parámetros:
    - f: La función de la EDO de la forma f(x, y).
    - x0: El valor inicial de x.
    - y0: El valor inicial de y (condición inicial).
    - x_end: El valor final de x hasta donde se quiere aproximar la solución.
    - step_size: El tamaño del paso (h).
    
    Retorna:
    - La aproximación de y en x_end.
    """

    x = x0
    y = y0

    while x < x_end:
        k1 = step_size * f(x, y)
        k2 = step_size * f(x + step_size / 2, y + k1 / 2)
        k3 = step_size * f(x + step_size / 2, y + k2 / 2)
        k4 = step_size * f(x + step_size, y + k3)
        y += (k1 + 2 * k2 + 2 * k3 + k4) / 6
        x += step_size
    
    return y

In [49]:
runge_kutta_4(f, 0, 1, 2, 0.1)

2.3221871812649915

In [50]:
%timeit runge_kutta_4(f, 0, 1, 2, 0.1)

35.3 µs ± 840 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [63]:
%%cython
cdef double f(double x, double y):
    return x*x*x - 2*y*y + 4

def runge_kutta_4(double x0, double y0, double x_end, double step_size):
    """
    Resuelve una ecuación diferencial ordinaria utilizando el método de Runge-Kutta de cuarto orden (RK4).
    
    Parámetros:
    - f: La función de la EDO de la forma f(x, y).
    - x0: El valor inicial de x.
    - y0: El valor inicial de y (condición inicial).
    - x_end: El valor final de x hasta donde se quiere aproximar la solución.
    - step_size: El tamaño del paso (h).
    
    Retorna:
    - La aproximación de y en x_end.
    """
    cdef:
        double x = x0
        double y = y0
        double k1, k2, k3, k4

    while x < x_end:
        k1 = step_size * f(x, y)
        k2 = step_size * f(x + step_size / 2, y + k1 / 2)
        k3 = step_size * f(x + step_size / 2, y + k2 / 2)
        k4 = step_size * f(x + step_size, y + k3)
        
        y += (k1 + 2 * k2 + 2 * k3 + k4) / 6
        x += step_size

    return y

Content of stdout:
_cython_magic_66c51bb51ca447c823918cba3d23dd3c5b0b9c57.c
   Creando biblioteca C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_66c51bb51ca447c823918cba3d23dd3c5b0b9c57.cp311-win_amd64.lib y objeto C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_66c51bb51ca447c823918cba3d23dd3c5b0b9c57.cp311-win_amd64.exp
Generando c¢digo
Generaci¢n de c¢digo finalizada

In [64]:
runge_kutta_4(0, 1, 2, 0.1)

2.3221871812649915

In [66]:
time = %timeit runge_kutta_4(0, 1, 2, 0.1)

525 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [72]:
def mejora(tiempo_original, tiempo_opt):
    porc = (tiempo_original - tiempo_opt)/ tiempo_original * 100
    return porc

In [76]:
t_original = 483E-3
mejora(t_original, 2.51E-3)

99.48033126293996